1. *Самостоятельно повторить tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [1]:
import pandas as pd

Наши новости

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt 
!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv

--2023-04-19 20:58:08--  https://docs.google.com/uc?export=download&confirm=t&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR
Распознаётся docs.google.com (docs.google.com)… 173.194.221.194, 2a00:1450:4010:c0a::c2
Подключение к docs.google.com (docs.google.com)|173.194.221.194|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 303 See Other
Адрес: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m8srjmchmi9ooh1tdga9a4hja5lisg4j/1681927050000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download&uuid=503503d9-3ffb-44df-9e91-ba2c0496962f [переход]
Предупреждение: в HTTP маски не поддерживаются.
--2023-04-19 20:58:08--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m8srjmchmi9ooh1tdga9a4hja5lisg4j/1681927050000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download&uuid=503503d9-3ffb-44df-9e91-ba2c0496962f
Распознаётся doc-0s-c0-docs.googleusercontent.com (doc-0s-c

In [3]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [4]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [5]:
!pip install razdel pymorphy2

     |████████████████████████████████| 55 kB 784 kB/s eta 0:00:01
     |████████████████████████████████| 8.2 MB 1.6 MB/s eta 0:00:01     |█████                           | 1.3 MB 1.6 MB/s eta 0:00:05     |██████▋                         | 1.7 MB 1.6 MB/s eta 0:00:05     |███████████▌                    | 2.9 MB 1.6 MB/s eta 0:00:04�██████████████▊               | 4.3 MB 1.6 MB/s eta 0:00:03
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=229508f6d82f52c53e622edc24529d61edebf361fbe8c06bc4fad6370c7cab6d
  Stored in directory: /home/krassa/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [6]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/krassa/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [9]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2023-04-19 21:01:51--  https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c
Распознаётся drive.google.com (drive.google.com)… 173.194.221.194, 2a00:1450:4010:c0a::c2
Подключение к drive.google.com (drive.google.com)|173.194.221.194|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 303 See Other
Адрес: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sonb9gkl0fsga1vkri8o1pv3pgiv22d7/1681927275000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download&uuid=82f5fc79-3c0b-4b29-a29a-157c40fcc901 [переход]
Предупреждение: в HTTP маски не поддерживаются.
--2023-04-19 21:01:51--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sonb9gkl0fsga1vkri8o1pv3pgiv22d7/1681927275000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download&uuid=82f5fc79-3c0b-4b29-a29a-157c40fcc901
Распознаётся doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-doc

In [10]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [11]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [12]:
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|                                                 | 0/27000 [00:00<?, ?it/s]/tmp/ipykernel_5214/4083466619.py:14: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
100%|████████████████████████████████████| 27000/27000 [00:29<00:00, 907.76it/s]


In [13]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [14]:
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|████████████████████████████████████| 27000/27000 [03:22<00:00, 133.27it/s]


А теперь в 3 строчки обучим нашу модель

In [15]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [16]:
N_topic = 20  # можно было менять

In [17]:
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=2)  # можно было менять

In [18]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [19]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [20]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(2, 0.23119773), (4, 0.18653147), (12, 0.07822792), (13, 0.4829177)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [21]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: произойти тело сотрудник местный мужчина обнаружить штат
topic_1: рак мероприятие программа сайт конкурс участник праздник
topic_2: пациент китай китайский сократиться россиянин тур таиланд
topic_3: россия правительство путин глава владимир закон право
topic_4: общество производитель кровь фотография следователь разместить юг
topic_5: млн британский великобритания место превысить озеро франция
topic_6: погибнуть трагедия ск бомба выдать огонь полицейский
topic_7: рубль цена газ ракета млн экономика мозг
topic_8: москва станция площадь столица торговый московский центр
topic_9: сша военный россия российский американский фонд журнал
topic_10: россия российский сторона система помощь территория сша
topic_11: адмирал инвестировать показ электроника скотт мак песок
topic_12: статья день наука научный университет жизнь советский
topic_13: ребёнок население гражданин семья всё день русский
topic_14: земля всё газета первый ru планета взрыв
topic_15: памятник область израиль конструкц

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [22]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [23]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.        , 0.02678616, 0.13248965, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.0928829 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.08874853, 0.40538794, 0.        , 0.24683237])

In [24]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.0,0.0,0.026778,0.132741,0.000000,0.000000,0.0,0.0,0.0,...,0.092887,0.0,0.000000,0.000000,0.000000,0.0,0.088767,0.40547,0.0,0.246484
1,4896,0.0,0.0,0.266157,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.710146,0.0,0.000000,0.00000,0.0,0.000000
2,4897,0.0,0.0,0.231199,0.000000,0.186535,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.078224,0.482917,0.000000,0.0,0.000000,0.00000,0.0,0.000000
3,4898,0.0,0.0,0.083536,0.108097,0.000000,0.062456,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.735644,0.0,0.000000,0.00000,0.0,0.000000
4,4899,0.0,0.0,0.000000,0.264582,0.000000,0.000000,0.0,0.0,0.0,...,0.297243,0.0,0.000000,0.000000,0.000000,0.0,0.187689,0.00000,0.0,0.226894


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [25]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [26]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [27]:
doc_dict[293672]

array([0.        , 0.12678628, 0.        , 0.        , 0.08116367,
       0.        , 0.        , 0.10660992, 0.15150975, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.10092058,
       0.        , 0.        , 0.        , 0.        , 0.41355497])

In [28]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [29]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.06894426, 0.00253403, 0.00741365, 0.19631172, 0.01600822,
       0.        , 0.        , 0.0629678 , 0.0118574 , 0.13756311,
       0.08092836, 0.        , 0.06943119, 0.03832455, 0.0475861 ,
       0.        , 0.        , 0.11172923, 0.        , 0.13301251])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [30]:
FUNC = np.mean

In [31]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.069834,0.073194,0.001684,0.171123,0.028670,0.02118,0.011259,0.043017,0.041855,...,0.071205,0.013875,0.032716,0.065652,0.066416,0.009809,0.003885,0.050716,0.000000,0.167724
1,u108690,0.091674,0.007837,0.000000,0.124768,0.007719,0.00000,0.003116,0.057677,0.009231,...,0.080219,0.000000,0.045917,0.069954,0.117288,0.000000,0.014243,0.027973,0.000000,0.217594
2,u108339,0.166952,0.018631,0.001771,0.132959,0.009012,0.00000,0.022790,0.063165,0.034683,...,0.121605,0.000000,0.037045,0.030145,0.076041,0.006060,0.010612,0.033517,0.015083,0.140295


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [32]:
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

--2023-04-19 21:22:55--  https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK
Распознаётся drive.google.com (drive.google.com)… 173.194.221.194, 2a00:1450:4010:c0a::c2
Подключение к drive.google.com (drive.google.com)|173.194.221.194|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 303 See Other
Адрес: https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ls95iqohtujktolh5j4hlsrusqegr2v7/1681928550000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download&uuid=c7928b60-8e91-47c1-b79f-ff26affa1487 [переход]
Предупреждение: в HTTP маски не поддерживаются.
--2023-04-19 21:22:56--  https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ls95iqohtujktolh5j4hlsrusqegr2v7/1681928550000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download&uuid=c7928b60-8e91-47c1-b79f-ff26affa1487
Распознаётся doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-doc

In [33]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [34]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.069834,0.073194,0.001684,0.171123,0.028670,0.02118,0.011259,0.043017,0.041855,...,0.013875,0.032716,0.065652,0.066416,0.009809,0.003885,0.050716,0.000000,0.167724,0
1,u108690,0.091674,0.007837,0.000000,0.124768,0.007719,0.00000,0.003116,0.057677,0.009231,...,0.000000,0.045917,0.069954,0.117288,0.000000,0.014243,0.027973,0.000000,0.217594,1
2,u108339,0.166952,0.018631,0.001771,0.132959,0.009012,0.00000,0.022790,0.063165,0.034683,...,0.000000,0.037045,0.030145,0.076041,0.006060,0.010612,0.033517,0.015083,0.140295,1


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [36]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [37]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [38]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.14538148, 0.09475264, 0.39154336, 0.0775828 , 0.0919439 ,
       0.04829628, 0.10124008, 0.00975223, 0.0711504 , 0.04643928])

In [39]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [40]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [41]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.29593400926819835, F-Score=0.711, Precision=0.768, Recall=0.661


In [42]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9547345775917204

In [43]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

/tmp/ipykernel_5214/1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.295934,0.710526,0.767773,0.661224,0.954735


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [44]:
FUNC = np.median

In [45]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.042432,0.088969,0.0,0.130486,0.006121,0.0,0.000000,0.000000,0.000000,...,0.013098,0.0,0.012679,0.023453,0.049565,0.0,0.000000,0.000000,0.000000,0.039383
1,u108690,0.063534,0.000000,0.0,0.127243,0.000000,0.0,0.000000,0.017704,0.005594,...,0.077697,0.0,0.041811,0.039392,0.109892,0.0,0.000000,0.000000,0.000000,0.251280
2,u108339,0.174615,0.000000,0.0,0.099853,0.000000,0.0,0.024886,0.046004,0.005548,...,0.115401,0.0,0.010157,0.032564,0.057061,0.0,0.007285,0.011049,0.013611,0.155236


In [46]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.042432,0.088969,0.0,0.130486,0.006121,0.0,0.000000,0.000000,0.000000,...,0.0,0.012679,0.023453,0.049565,0.0,0.000000,0.000000,0.000000,0.039383,0
1,u108690,0.063534,0.000000,0.0,0.127243,0.000000,0.0,0.000000,0.017704,0.005594,...,0.0,0.041811,0.039392,0.109892,0.0,0.000000,0.000000,0.000000,0.251280,1
2,u108339,0.174615,0.000000,0.0,0.099853,0.000000,0.0,0.024886,0.046004,0.005548,...,0.0,0.010157,0.032564,0.057061,0.0,0.007285,0.011049,0.013611,0.155236,1


In [47]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [48]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [49]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.15693685, 0.08732769, 0.44936413, 0.07216443, 0.16276508,
       0.04534599, 0.0318609 , 0.01065242, 0.02812988, 0.08706345])

In [50]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.24825894164414358, F-Score=0.786, Precision=0.734, Recall=0.845


In [51]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9753404267689982

In [52]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

/tmp/ipykernel_5214/1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.295934,0.710526,0.767773,0.661224,0.954735
1,median,0.248259,0.785579,0.734043,0.844898,0.97534



3. Повторить п.2, но используя уже не медиану, а max


In [53]:
FUNC = np.max

In [54]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.181264,0.134439,0.010102,0.442625,0.081164,0.127079,0.049729,0.151493,0.151510,...,0.214949,0.083252,0.142651,0.187109,0.198444,0.058853,0.023309,0.304297,0.000000,0.514025
1,u108690,0.272069,0.036135,0.000000,0.172792,0.046316,0.000000,0.018699,0.190142,0.031342,...,0.131403,0.000000,0.109922,0.179873,0.163976,0.000000,0.085459,0.137487,0.000000,0.309089
2,u108339,0.287615,0.086593,0.010627,0.256235,0.033503,0.000000,0.057842,0.218546,0.118557,...,0.172222,0.000000,0.107626,0.059818,0.182798,0.036359,0.026723,0.098255,0.045184,0.266285


In [55]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.181264,0.134439,0.010102,0.442625,0.081164,0.127079,0.049729,0.151493,0.151510,...,0.083252,0.142651,0.187109,0.198444,0.058853,0.023309,0.304297,0.000000,0.514025,0
1,u108690,0.272069,0.036135,0.000000,0.172792,0.046316,0.000000,0.018699,0.190142,0.031342,...,0.000000,0.109922,0.179873,0.163976,0.000000,0.085459,0.137487,0.000000,0.309089,1
2,u108339,0.287615,0.086593,0.010627,0.256235,0.033503,0.000000,0.057842,0.218546,0.118557,...,0.000000,0.107626,0.059818,0.182798,0.036359,0.026723,0.098255,0.045184,0.266285,1


In [56]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [57]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [58]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([9.06386701e-02, 3.04317674e-03, 6.99678369e-01, 5.04451188e-02,
       4.14503716e-01, 2.14769546e-03, 3.08798708e-02, 6.26558188e-04,
       1.43336678e-02, 1.41907131e-01])

In [59]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.353421236157998, F-Score=0.770, Precision=0.756, Recall=0.784


In [60]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9663538577824291

In [61]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

/tmp/ipykernel_5214/1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.295934,0.710526,0.767773,0.661224,0.954735
1,median,0.248259,0.785579,0.734043,0.844898,0.97534
2,amax,0.353421,0.769539,0.755906,0.783673,0.966354


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [64]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [65]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [66]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.median(user_vector, axis=0)
    return user_vector

In [67]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|███████████████████████████████████████| 8000/8000 [01:49<00:00, 73.38it/s]


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.347756,0.755642,0.0,1.179222,0.052646,0.0,0.000000,0.000000,0.000000,...,0.121736,0.0,0.109055,0.201722,0.434010,0.0,0.000000,0.000000,0.000000,0.338732
1,u108690,0.563119,0.000000,0.0,1.081588,0.000000,0.0,0.000000,0.147436,0.045845,...,0.681372,0.0,0.367384,0.346823,0.914467,0.0,0.000000,0.000000,0.000000,2.083066
2,u108339,1.540269,0.000000,0.0,0.878553,0.000000,0.0,0.210848,0.416296,0.047717,...,0.932896,0.0,0.085095,0.269570,0.471154,0.0,0.067706,0.102692,0.112815,1.319599


In [68]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.347756,0.755642,0.0,1.179222,0.052646,0.0,0.000000,0.000000,0.000000,...,0.0,0.109055,0.201722,0.434010,0.0,0.000000,0.000000,0.000000,0.338732,0
1,u108690,0.563119,0.000000,0.0,1.081588,0.000000,0.0,0.000000,0.147436,0.045845,...,0.0,0.367384,0.346823,0.914467,0.0,0.000000,0.000000,0.000000,2.083066,1
2,u108339,1.540269,0.000000,0.0,0.878553,0.000000,0.0,0.210848,0.416296,0.047717,...,0.0,0.085095,0.269570,0.471154,0.0,0.067706,0.102692,0.112815,1.319599,1


In [69]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [70]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [71]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([1.10033456e-01, 2.07421320e-03, 9.27720972e-01, 1.11982477e-02,
       1.97762268e-01, 6.76576963e-04, 4.20181934e-04, 1.33820619e-07,
       1.96085763e-05, 2.17113710e-02])

In [72]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.4668298463998236, F-Score=0.874, Precision=0.897, Recall=0.853


In [73]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9886644572358858

In [74]:
metrics_df = metrics_df.append({
    'model': 'idf_median',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

/tmp/ipykernel_5214/1015548053.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({



5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [75]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.295934,0.710526,0.767773,0.661224,0.954735
1,median,0.248259,0.785579,0.734043,0.844898,0.97534
2,amax,0.353421,0.769539,0.755906,0.783673,0.966354
3,idf_median,0.46683,0.874477,0.896996,0.853061,0.988664


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

Лучшую метрику roc-auc показала модель, где используется алгоритм взвешивания весов TfidfVectorizer. Видимо взвешивание весов слов этим алгоритмом + использование медианы (которая показала лучший результат по сравнению со средним и максимальным) позволяет добиться максимальной метрики. 